In [3]:
#beware of whether the functions will alter the variables

#def lincom(<list of dictionaries>, <list of coeff>) e.g. lincom([u,v,w], [3,4,-1]) => return dictionary: 3u + 4v - w
def lincom(l, c):
    r = {}
    n = len(c) #len(l) should equal to len(c)
    for i in range(n):
        r = plus(r, scal(c[i], l[i]))
    return r

def show(d):
    for k in d:
        print(k, '------', d[k]) 

def plus(u, v): 
    result = v.copy()
    for k in u:
        if k in result:
            result[k] += u[k]
        else:
            result[k] = u[k]
    return result

def plus1(u, v): #v will CHANGE to u+v; u won't change
    for k in u:
        if k in v:
            v[k] += u[k]
        else:
            v[k] = u[k]
            
def scal(n, d): #d is a dictionary; return scalar product of d by n
    r = d.copy()
    for k in r:
        r[k] *= n
    return r

def scal1(n, d): #d will CHANGE
    for k in d:
        d[k] *= n
        
def concat(u, v): #u,v are dicts with xy-string/z-value-tuple keys; return uv
                    #watch out when u, v are empty dicts
    r = {}
    for k in u:
        for l in v:           
            r[k + l] = u[k] * v[l] #k + l joins two strings or tuples
    return r

def mono_sh(s, t): #s, t are xy-strings; return dictionary
    if s == '':
        return {t: 1}
    if t == '':
        return {s: 1}
    
    s1 = s[0 : len(s) - 1]
    t1 = t[0 : len(t) - 1]
    a = concat(mono_sh(s, t1), {t[-1]: 1})
    b = concat(mono_sh(s1, t), {s[-1]: 1})
    plus1(a, b) #b will change to a+b
    return b

def mono_st_new(s, t): #s, t are TUPLES
    if s == ():
        return {t: 1} #includes the case when both s, t are (). In that case, would return {(): 1}
    if t == ():
        return {s: 1}
    
    s1 = s[0 : len(s) - 1] #s1 is still a tuple
    t1 = t[0 : len(t) - 1]
    a = concat(mono_st_new(s, t1), {(t[-1],): 1}) #t[-1] is a number, not tuple
    b = concat(mono_st_new(s1, t), {(s[-1],): 1})
    c = concat(mono_st_new(s1, t1), {(s[-1] + t[-1],): 1})
    plus1(a, b) #b will change to a+b
    plus1(b, c) #c will change to a+b+c
    return c    

def mono_st(head, s, t, dic): #head, s, t are lISTS
    if len(s) == 0:
        r = tuple(head + t) #join two lists AND TRANSFORM TO TUPLE <- DOESN'T HAVE TO. MODIFY THIS PART
                            #dictionary cannot use lists as keys because they are changable
        if r in dic:
            dic[r] += 1
        else:
            dic[r] = 1
        return 
    
    if len(t) == 0:
        r = tuple(head + s)
        if r in dic:
            dic[r] += 1
        else:
            dic[r] = 1
        return 
    
    heads = head + [s[0]]
    headt = head + [t[0]]
    headspt = head + [s[0]+t[0]]
    '''
    s_copy = list(s) #can't use s.copy()
    s.pop(0) #now s does not have the first number
    mono_st(heads, s, t, dic)
    t.pop(0) #pop also makes trouble. don't fucking know why
    mono_st(headt, s_copy, t, dic)
    mono_st(headspt, s, t, dic)
    '''
    mono_st(heads, s[1:len(s)], t, dic)
    mono_st(headt, s, t[1:len(t)], dic)
    mono_st(headspt, s[1:len(s)], t[1:len(t)], dic)

def xysh(u, v): #u,v are dicts. But their keys are strings of xy-binary words
    result = {}
    for u_mono in u: #mono means monomial
        for v_mono in v:
            r = mono_sh(u_mono, v_mono)            
            scal1(u[u_mono]*v[v_mono], r)
            plus1(r, result)
    return result #the keys of result are also xy-binary words

def toxy(d): #d: dictionary whose keys are tuples. Transform d to another dictionary r whose keys are strings of xy-binary words
    r = {}
    for k in d:
        s = ''
        for n in k: #e.g. k = (2,3,9) then n is integer 2 -> 3 -> 9
            s += 'y'
            for i in range(n-1): #append 'y' with (n-1) 'x'
                s += 'x'
        r[s] = d[k]
    return r

def toz(d): #reverse of toxy.
    # 'yxxxyxyyyx': place of y: 0, 4, 6, 7, 8. Don't forget the last one: z2. 2 = len - 8 = 10 - 8
    r = {}
    for k in d: #e.g. k = 'yxxxyxyyyx'
        l = []
        for i in range(len(k)): #i=0 ~ len(k)-1
            if k[i] == 'y':
                l.append(i) #l records position of 'y'
                
        l.append(len(k)) #Don't forget the last one; now l = [0,4,6,7,8,10]
        ll = []
        i = 0
        while i <= len(l) - 2:
            ll.append(l[i+1] - l[i])
            i += 1
        
        t = tuple(ll)
        r[t] = d[k]
    return r

def sh(u, v): #u, v are dictionary whose keys are z-value tuples
    u1 = toxy(u)
    v1 = toxy(v)
    return toz(xysh(u1, v1)) #also return dictionary whose keys are z-value tuples
 
def st(u, v): #z-value stuffle
    result = {}
    for u_mono in u: 
        for v_mono in v:
            u_mono_l = list(u_mono) #u_mono is a TUPLE
            v_mono_l = list(v_mono)
            r = {}; mono_st([], u_mono_l, v_mono_l, r) #r = u_mono st v_mono
            scal1(u[u_mono]*v[v_mono], r)
            plus1(r, result)
    return result

def st_new(u, v): #z-value stuffle
    result = {}
    for u_mono in u: 
        for v_mono in v:
            r = mono_st_new(u_mono, v_mono)
            scal1(u[u_mono]*v[v_mono], r)
            plus1(r, result)
    return result

def reg_sh(d):
    r = {}
    for k in d:
        if k[-1] != 1:
            #simply add k into r
            if k in r:
                r[k] += d[k]
            else:
                r[k] = d[k]
        else: 
            #add reg_sh(k) into r
            m = 0
            ones = []
            while k[-(m+1)] == 1:
                m += 1
                ones = ones + [1]
            k_list = list(k)
            k_len = len(k_list)
            w0 = k_list[0 : k_len - m] #k = w0 . {1}^m (' . ' is concatenation)
            w0[-1] -= 1 #w0 becomes w0'
            reg_k_1 = sh( {tuple(ones): (-1)**m}, {tuple(w0): 1} )
            
            reg_k = {}
            for key in reg_k_1: #concatenate reg_k with x at its end
                key_x = list(key); key_x[-1] += 1; key_x = tuple(key_x)
                reg_k[key_x] = reg_k_1[key]
                
            r = plus(r, scal(d[k],reg_k)) #rewrite with plus1, scal1          
    return r    

import data

def mono_diff_del(n, s): 
    if s == 'x':
        return data.del_x[n]
    if s == 'y':             
        return scal(-1, data.del_x[n])
    
    s1 = s[0 : len(s) - 1]
    if s[-1] == 'x':
        a = concat(mono_diff_del(n, s1), {'x': 1})
        b = concat({s1: 1}, data.del_x[n])
        plus1(a, b) #b changes to a+b
        return b
    else: 
        a = concat(mono_diff_del(n, s1), {'y': 1})
        b = concat({s1: -1}, data.del_x[n])
        plus1(a, b)
        return b
        
'''
def diff_del(n, d, delx): #the keys of d are xy-strings
    
    for k in d:
        k1 = k[0 : len(k)-1] #taking away the last character
        if k[-1] == 'x':
            a = concat(diff_del(n, k1, delx), {'x': 1})
            b = plus(a, concat(k1, delx[n]))
            plus1(r, b)
        else:          

'''
        
def xyDict_to_list(w, d): #the keys of d are xy-strings BELONGING TO H^0 and have to be of the SAME WEIGHT w = len(key)
    #create list with 2^(w-2) zeros
    l = []
    for i in range( 2**(w-2) ):
        l.append(0)
    
    for k in d:
        n = 0 #convert k to its index in l
        i = w-2 #second to last character of string k   
        while i >= 1: #traverse back till the second character
            if k[i] == 'y':
                n += 2**(w-2-i)
            i -= 1
        l[n] = d[k]
    return l
    

In [17]:
w = {
    (1,2): 1
}
y = {(1,): 1}
show(reg_sh(st_new( y, w )))

((1, 3), '------', 1)
((2, 2), '------', 1)
((1, 1, 2), '------', -1)


In [21]:
u = {
    (2,2): 9,
    (3,3): 4/3
}
v = {
    (5,5): 8,
    (2,): 16/5 #one-element tuple needs that comma
}
w = {
    (2,): 1,
    (1,2): 1
}
r = sh( u, st(v, w) )
for k in r:
    print(k, '------', r[k])

((4, 3, 4, 3, 2, 3), '------', 128)
((5, 2, 3, 4, 3), '------', 1008)
((7, 4, 3, 2), '------', 144)
((2, 4, 1, 3, 9), '------', 896/3)
((3, 3, 5, 3, 3), '------', 576)
((2, 5, 5, 3, 4), '------', 864)
((2, 5, 1, 6, 2, 3), '------', 800)
((1, 1, 4, 3, 4, 6), '------', 1440)
((1, 3, 4, 4, 2, 3), '------', 432)
((3, 3, 2, 4, 6), '------', 320)
((1, 2, 1, 6, 2, 7), '------', 4800)
((1, 5, 2, 4, 7), '------', 1920)
((4, 4, 2, 5, 2), '------', 1296)
((1, 8, 4, 5), '------', 4032)
((5, 1, 2, 1, 8), '------', 2016)
((5, 3, 4, 4), '------', 1296)
((3, 1, 3, 1, 3), '------', 512/15)
((1, 5, 3, 5, 2), '------', 2592)
((1, 2, 4, 4, 7), '------', 960)
((4, 2, 7, 1, 3, 2), '------', 320)
((4, 2, 5, 2, 3), '------', 720)
((4, 3, 1, 2, 5, 4), '------', 320)
((4, 4, 5, 1, 3), '------', 864)
((1, 1, 7, 3, 2, 5), '------', 480)
((5, 1, 6, 1, 2, 2), '------', 2160)
((4, 4, 4, 3, 4), '------', 384)
((4, 2, 2, 3, 4, 4), '------', 128)
((5, 3, 2, 6, 2), '------', 160)
((3, 5, 6, 2, 3), '------', 864)
((5, 1,

((5, 3, 1, 1, 2, 7), '------', 480)
((1, 7, 2, 2, 7), '------', 9120)
((3, 3, 1, 8, 2), '------', 504)
((2, 4, 3, 5, 5), '------', 64)
((2, 3, 5, 2, 2, 5), '------', 128)
((2, 2, 2, 5, 5, 3), '------', 352/3)
((1, 1, 4, 3, 2), '------', 384/5)
((4, 3, 1, 1, 5, 3), '------', 864)
((1, 6, 4, 2, 1, 3), '------', 1440)
((4, 4, 1, 1, 9), '------', 2688)
((3, 4, 3, 3, 1, 3), '------', 432)
((2, 5, 2, 1, 2, 5), '------', 360)
((5, 2, 3, 2, 4), '------', 216)
((3, 3, 3, 6, 2), '------', 1728)
((1, 1, 7, 5, 3), '------', 1728)
((5, 5, 3, 2, 2, 2), '------', 32/3)
((2, 1, 3, 3, 2), '------', 128/5)
((5, 4, 4, 5), '------', 864)
((1, 1, 2, 5, 8), '------', 1512)
((1, 4, 2, 7, 3), '------', 864)
((2, 1, 5, 2, 3, 6), '------', 2240/3)
((1, 1, 6, 7, 4), '------', 2880)
((1, 3, 1, 3, 5, 6), '------', 1280/3)
((4, 4, 1, 3, 3, 4), '------', 96)
((2, 4, 8, 3, 2), '------', 224)
((2, 4, 4, 3, 6), '------', 640)
((5, 3, 4, 2, 2, 3), '------', 544/3)
((5, 4, 4, 2, 2, 2), '------', 32)
((1, 3, 2, 1, 2), '--

((2, 4, 2, 5, 1, 3), '------', 1008)
((3, 5, 1, 5, 5), '------', 448)
((2, 2, 2, 3, 2), '------', 192/5)
((5, 3, 2, 1, 8), '------', 224/3)
((5, 5, 3, 3, 3), '------', 256/3)
((2, 5, 4, 3, 5), '------', 864)
((4, 3, 3, 4, 5), '------', 448)
((2, 1, 5, 6, 2, 3), '------', 160)
((3, 1, 3, 5, 2, 5), '------', 256/3)
((4, 2, 3, 1, 6), '------', 1440)
((7, 1, 6, 2), '------', 720)
((2, 1, 4, 3, 6, 3), '------', 320)
((4, 3, 1, 2, 4, 3), '------', 576)
((1, 2, 3, 3, 7, 3), '------', 160)
((5, 4, 1, 9), '------', 1792)
((1, 5, 3, 2, 8), '------', 1792)
((1, 2, 6, 3, 4, 3), '------', 1280)
((6, 1, 3, 7), '------', 1728)
((1, 4, 4, 4, 4, 2), '------', 96)
((1, 1, 5, 3, 2, 7), '------', 4320)
((3, 4, 1, 4, 2, 3), '------', 432)
((5, 5, 3, 1, 3, 2), '------', 64/3)
((3, 2, 5, 5, 1, 3), '------', 320/3)
((2, 1, 6, 4, 6), '------', 4000/3)
((2, 6, 1, 6, 2, 2), '------', 1600)
((5, 3, 1, 6, 2), '------', 1440)
((5, 1, 2, 3, 3, 5), '------', 1312/3)
((3, 1, 5, 4, 5), '------', 64)
((2, 2, 4, 2, 6), '

((2, 6, 3, 3, 5), '------', 1536)
((1, 5, 4, 2, 2, 3), '------', 288)
((3, 3, 4, 6, 3), '------', 1984/3)
((3, 3, 1, 6, 4, 2), '------', 320)
((1, 4, 4, 5, 3), '------', 1440)
((5, 4, 3, 2, 3, 2), '------', 32)
((2, 4, 1, 5, 3, 4), '------', 224)
((1, 5, 3, 4, 3), '------', 1152)
((1, 5, 1, 6, 4, 2), '------', 1280)
((3, 4, 5, 2, 4), '------', 480)
((1, 2, 8, 1, 7), '------', 11200)
((5, 2, 5, 7), '------', 960)
((3, 2, 4, 7, 3), '------', 128)
((5, 4, 2, 5, 3), '------', 352)
((3, 1, 6, 6, 3), '------', 1280/3)
((1, 3, 2, 3, 8), '------', 504)
((1, 5, 2, 2, 1, 6), '------', 2880)
((4, 2, 6, 4), '------', 1080)
((3, 5, 6, 3, 2), '------', 288)
((3, 4, 2, 4, 1, 3), '------', 432)
((4, 3, 5, 2, 3), '------', 1008)
((3, 3, 1, 2, 4, 4), '------', 648)
((4, 4, 2, 4, 3), '------', 864)
((4, 2, 2, 2, 5, 2), '------', 504)
((1, 7, 3, 5, 1, 2), '------', 2240)
((4, 3, 5, 6), '------', 3200/3)
((1, 2, 4, 3, 9), '------', 8960/3)
((4, 3, 2, 3, 3, 4), '------', 64)
((5, 1, 1, 2, 7, 3), '------', 4

((1, 4, 6, 2, 5), '------', 640)
((1, 6, 3, 3, 5), '------', 7040/3)
((4, 2, 3, 3, 5), '------', 144)
((3, 2, 7, 5, 2), '------', 224)
((3, 2, 3, 6, 1, 2), '------', 360)
((2, 1, 6, 5, 5), '------', 640)
((3, 3, 3, 4, 6), '------', 320/3)
((1, 5, 3, 3, 3, 4), '------', 160)
((1, 1, 7, 3, 7), '------', 15360)
((5, 2, 4, 4, 1, 3), '------', 384)
((1, 2, 3, 3, 2), '------', 256/5)
((1, 7, 2, 3, 5), '------', 2880)
((4, 4, 2, 5, 1, 3), '------', 320)
((4, 2, 2, 5, 6), '------', 1280/3)
((2, 2, 4, 3, 3, 5), '------', 128/3)
((5, 2, 2, 1, 7), '------', 72)
((5, 1, 3, 5, 2, 3), '------', 896/3)
((5, 4, 3, 3, 1, 3), '------', 128/3)
((5, 2, 4, 4, 3), '------', 448)
((3, 3, 4, 5, 3), '------', 416)
((1, 1, 6, 5, 4), '------', 3240)
((1, 1, 5, 6, 4, 2), '------', 960)
((1, 4, 4, 6, 3), '------', 640)
((1, 7, 3, 2, 6), '------', 7200)
((2, 3, 5, 2, 5), '------', 864)
((2, 4, 2, 3), '------', 384/5)
((2, 1, 6, 4, 2, 4), '------', 160)
((3, 5, 4, 5), '------', 1152)
((5, 3, 3, 5, 3), '------', 1792

...

((3, 3, 2, 6, 1, 2), '------', 1080)
((1, 5, 3, 6, 3), '------', 5120/3)
((5, 3, 7, 2), '------', 144)
((1, 5, 2, 4, 4, 3), '------', 832)
((4, 4, 5, 2, 2), '------', 432)
((6, 2, 3, 8), '------', 672)
((5, 1, 1, 2, 8), '------', 1512)
((7, 3, 3, 3), '------', 288)
((1, 1, 6, 3), '------', 128)
((1, 3, 4, 4, 6), '------', 1920)
((5, 2, 1, 2, 6, 3), '------', 160)
((1, 4, 2, 4, 1, 7), '------', 960)
((2, 3, 4, 1, 7, 2), '------', 480)
((2, 4, 7, 1, 2, 3), '------', 800)
((3, 4, 4, 1, 7), '------', 1088)
((3, 3, 6, 1, 3, 3), '------', 320/3)
((2, 4, 1, 2, 6, 4), '------', 320)
((1, 1, 6, 3, 3, 5), '------', 3520)
((2, 1, 8, 5), '------', 1008)
((1, 1, 3, 3, 4, 5), '------', 1296)
((3, 5, 1, 5, 3), '------', 1152)
((5, 2, 2, 3, 2, 5), '------', 32/3)
((3, 1, 6, 3, 5), '------', 640/3)
((1, 1, 3, 4, 2, 6), '------', 4320)
((5, 5, 1, 1, 3, 2), '------', 432)
((1, 7, 4, 2, 1, 4), '------', 960)
((3, 5, 4, 2, 3, 2), '------', 224/3)
((3, 1, 4, 2, 6, 3), '------', 320/3)
((3, 5, 5, 5), '------

((1, 3, 3, 3, 4, 5), '------', 512)
((1, 2, 4, 3, 3, 6), '------', 6080/3)
((3, 1, 3, 2, 5, 5), '------', 64/3)
((5, 2, 1, 2, 2, 5), '------', 504)
((5, 1, 3, 2, 5), '------', 1440)
((1, 3, 1, 7, 5, 2), '------', 640)
((5, 1, 4, 4, 1, 2), '------', 1296)
((1, 2, 2, 3, 3, 6), '------', 1800)
((1, 6, 4, 4, 1, 3), '------', 1920)
((3, 2, 4, 8, 2), '------', 224)
((5, 1, 2, 6, 4), '------', 640)
((3, 4, 1, 5, 3, 3), '------', 576)
((2, 1, 6, 8), '------', 1008)
((1, 6, 1, 5, 3), '------', 2880)
((1, 6, 3, 2, 7), '------', 15200/3)
((5, 2, 1, 2, 5, 2), '------', 504)
((5, 1, 3, 7, 3), '------', 1312/3)
((1, 4, 3, 3, 8), '------', 3136/3)
((4, 2, 2, 4, 6), '------', 320)
((1, 6, 3, 5, 4), '------', 2112)
((4, 2, 3, 6, 1, 3), '------', 320/3)
((6, 1, 7, 5), '------', 1920)
((5, 2, 4, 1, 3, 2), '------', 144)
((2, 1, 1, 5, 5, 3), '------', 432)
((5, 2, 1, 5, 6), '------', 1280/3)
((5, 2, 2, 2, 5, 3), '------', 352/3)
((1, 4, 3, 1, 3, 5), '------', 1440)
((1, 3, 3, 3, 9), '------', 3584/3)
((2,

((2, 6, 4, 2, 3, 2), '------', 160)
((5, 1, 2, 5, 3), '------', 1440)
((4, 3, 5, 2, 5), '------', 64)
((1, 7, 1, 6, 3), '------', 6400)
((3, 3, 1, 3, 1, 6), '------', 1440)
((5, 3, 4, 1, 1, 3), '------', 1296)
((2, 3, 1, 1, 4), '------', 576/5)
((1, 2, 2, 5, 6, 3), '------', 1760/3)
((1, 3, 2, 3, 6, 2), '------', 360)
((5, 1, 3, 4, 4), '------', 648)
((2, 5, 2, 4, 2, 4), '------', 160)
((1, 1, 6, 1, 5, 3), '------', 4320)
((3, 3, 1, 9, 3), '------', 896/3)
((2, 4, 7, 4, 2), '------', 480)
((2, 5, 6, 1, 3), '------', 2880)
((1, 1, 4, 2, 3), '------', 1152/5)
((1, 6, 1, 3, 6), '------', 3600)
((4, 3, 2, 1, 9), '------', 4480/3)
((3, 3, 3, 7, 3), '------', 1184/3)
((3, 1, 2, 5, 5, 3), '------', 416/3)
((1, 2, 3, 4, 7), '------', 864)
((2, 1, 3, 2, 3), '------', 384/5)
((2, 6, 5, 4, 2), '------', 1248)
((2, 2, 5, 9), '------', 896/3)
((1, 5, 1, 2, 4, 4), '------', 2592)
((2, 6, 1, 6, 2), '------', 4320)
((1, 4, 2, 8, 4), '------', 672)
((1, 4, 4, 1, 7, 2), '------', 1920)
((2, 4, 8, 2, 3),

((6, 2, 2, 2, 5), '------', 1080)
((2, 1, 6, 2, 1, 7), '------', 1600)
((1, 4, 2, 3), '------', 768/5)
((5, 2, 2, 4, 3), '------', 1008)
((2, 7, 3, 5, 2), '------', 1568)
((3, 3, 2, 5, 3), '------', 1008)
((2, 5, 6, 2, 4), '------', 1440)
((3, 3, 7, 1, 2, 3), '------', 800)
((1, 1, 4, 6, 2, 5), '------', 960)
((1, 2, 2, 1, 3), '------', 5184/5)
((2, 5, 1, 5, 3, 3), '------', 960)
((2, 1, 2, 7, 7), '------', 320)
((1, 1, 2, 7, 1, 7), '------', 14400)
((2, 1, 5, 2, 5, 4), '------', 288)
((5, 3, 6, 5), '------', 640)
((1, 6, 2, 1, 6, 3), '------', 12800/3)
((1, 2, 4, 4, 3, 5), '------', 1024)
((4, 3, 2, 7, 3), '------', 1696/3)
((5, 3, 1, 7, 3), '------', 800/3)
((3, 6, 1, 2, 7), '------', 3520)
((1, 4, 4, 2, 2, 6), '------', 960)
((4, 5, 5, 2, 3), '------', 192)
((3, 4, 7, 3, 2), '------', 352)
((4, 3, 1, 2, 4, 5), '------', 448)
((1, 5, 5, 1, 3, 2), '------', 576)
((1, 3, 6, 1, 6, 2), '------', 1600/3)
((2, 4, 3, 1, 4, 5), '------', 256)
((3, 4, 4, 3, 3), '------', 864)
((3, 5, 2, 1, 5,

((4, 2, 1, 3, 5, 4), '------', 288)
((1, 6, 4, 3, 3), '------', 2304)
((1, 2, 3, 3, 5, 5), '------', 2464/3)
((2, 6, 4, 4, 1, 2), '------', 480)
((1, 2, 5, 4, 5), '------', 3024)
((1, 4, 2, 5, 3, 4), '------', 224)
((5, 2, 1, 2, 3, 6), '------', 320)
((1, 1, 5, 4, 2, 6), '------', 4320)
((4, 4, 5, 5), '------', 768)
((2, 3, 3, 1, 8), '------', 1512)
((1, 8, 6, 4), '------', 1344)
((3, 4, 4, 4, 2, 2), '------', 288)
((4, 3, 1, 4, 5, 2), '------', 384)
((4, 2, 8, 1, 4), '------', 1344)
((2, 5, 3, 5, 2), '------', 2880)
((2, 1, 6, 3, 2, 5), '------', 160/3)
((3, 5, 2, 5, 3), '------', 2464/3)
((1, 5, 1, 8, 4), '------', 2688)
((3, 4, 3, 6, 1, 2), '------', 3200/3)
((1, 7, 1, 1, 4, 5), '------', 2880)
((2, 1, 8, 4, 4), '------', 224)
((4, 4, 1, 1, 4, 5), '------', 288)
((1, 4, 2, 3, 2, 7), '------', 480)
((2, 2, 2, 5, 8), '------', 1120/3)
((5, 2, 6, 2, 2), '------', 1584)
((3, 3, 6, 3, 2), '------', 864)
((3, 4, 1, 3, 5), '------', 864)
((1, 6, 1, 2, 4, 3), '------', 2880)
((2, 6, 3, 3, 2

In [10]:
u = {
    (2,): 1,
    (1,2): 1
}
v = {
    (2,): 1
}
w = sh(u,v)
for k in w:
    print(k, '------', w[k])

((1, 2, 2), '------', 3)
((1, 1, 3), '------', 6)
((1, 3), '------', 4)
((2, 1, 2), '------', 1)
((2, 2), '------', 2)


In [29]:
u = {
    "yyx" : 3,
    "yxx" : 2/3
}

v = {
    "yx" : 7,
    "yyxyx" : 3,
    "yxxyx" : 2/3
}
xyDict_to_list(3,u)

[2/3, 3]

In [143]:
u = {
    (2,): 1,
    (1,2): 1
}
v = {
    (2,): 1
}
show( lincom( [u,v], [1,-1/4] ) )

((1, 2), '------', 1)
((2,), '------', 3/4)


In [88]:
l = []; l.append(3); l

[3]

In [139]:
l = [{"yxx": 3, "yx": -1}, {"yyxxx": 7}]; l[0]['yxx']

3

In [153]:
A = Matrix([[1,2,3,4],
            [5,6,7,8]])
A[0][3]

4

In [166]:
a = [3,2,1]; d = {tuple(a) : -2};

In [173]:
(-1)**23445645765876812121

-1

In [47]:
reg_sh( plus( st({(1,): 1},{(1,2): 1}) , st({(1,): 1},{(3,): 1}) ) )

{(1, 1, 2): -1, (1, 3): 0, (2, 2): 0, (4,): 1}

In [6]:
z5 = {(5,): 1}
z2 = {(2,): 1}
show(sh(z5, z2))

((3, 4), '------', 3)
((1, 6), '------', 10)
((2, 5), '------', 5)
((5, 2), '------', 1)
((4, 3), '------', 2)


In [8]:
z4 = {(4,): 1}
z3 = {(3,): 1}
show(sh(z4, z3))

((1, 6), '------', 20)
((2, 5), '------', 10)
((3, 4), '------', 4)
((4, 3), '------', 1)


In [18]:
u = {(2,3,3,13,4): 1}
r = sh(z3, u)
v = {(2,3,3,12,4): 1}
s = sh(z4, v)
'''
a = []
for k in r: #k: tuple
    location = 15 - k[-2] #where tuple k should be stored in a
    if a[location]
show(sh(z3, u))
'''

i = 15
while i >= 1:        
    for k in r:
        if k[-2] == i:
            print (k, '---', r[k])
    i -= 1


((2, 2, 2, 3, 15, 4), '---', 91)
((2, 3, 3, 1, 15, 4), '---', 182)
((2, 1, 3, 3, 15, 4), '---', 182)
((2, 3, 2, 2, 15, 4), '---', 91)
((1, 2, 3, 3, 15, 4), '---', 182)
((2, 3, 1, 3, 15, 4), '---', 182)
((2, 3, 2, 2, 14, 5), '---', 52)
((2, 3, 1, 4, 14, 4), '---', 78)
((2, 3, 2, 3, 14, 4), '---', 39)
((1, 2, 3, 3, 14, 5), '---', 104)
((2, 3, 3, 2, 14, 4), '---', 91)
((2, 2, 3, 3, 14, 4), '---', 52)
((2, 3, 3, 1, 14, 5), '---', 104)
((2, 1, 3, 3, 14, 5), '---', 104)
((2, 2, 2, 4, 14, 4), '---', 39)
((2, 1, 3, 4, 14, 4), '---', 78)
((2, 2, 2, 3, 14, 5), '---', 52)
((1, 2, 4, 3, 14, 4), '---', 78)
((2, 3, 1, 3, 14, 5), '---', 104)
((1, 3, 3, 3, 14, 4), '---', 52)
((1, 2, 3, 4, 14, 4), '---', 78)
((2, 1, 4, 3, 14, 4), '---', 78)
((2, 3, 1, 3, 13, 6), '---', 20)
((1, 2, 3, 4, 13, 5), '---', 24)
((2, 3, 3, 3, 13, 4), '---', 71)
((1, 2, 3, 5, 13, 4), '---', 12)
((2, 1, 4, 3, 13, 5), '---', 24)
((1, 3, 3, 4, 13, 4), '---', 12)
((2, 1, 5, 3, 13, 4), '---', 12)
((2, 3, 1, 4, 13, 5), '---', 24)
((

In [21]:
u = {(2,3,3,13,4): 1}
r = sh(z3, u)
'''
a = []
for k in r: #k: tuple
    location = 15 - k[-2] #where tuple k should be stored in a
    if a[location]
show(sh(z3, u))
'''

#the following code is stupid
i = 15
while i >= 1:        
    for k in r:
        if k[-2] == i:
            print (k, '---', r[k])
    i -= 1

((2, 2, 2, 3, 15, 4), '---', 91)
((2, 3, 3, 1, 15, 4), '---', 182)
((2, 1, 3, 3, 15, 4), '---', 182)
((2, 3, 2, 2, 15, 4), '---', 91)
((1, 2, 3, 3, 15, 4), '---', 182)
((2, 3, 1, 3, 15, 4), '---', 182)
((2, 3, 2, 2, 14, 5), '---', 52)
((2, 3, 1, 4, 14, 4), '---', 78)
((2, 3, 2, 3, 14, 4), '---', 39)
((1, 2, 3, 3, 14, 5), '---', 104)
((2, 3, 3, 2, 14, 4), '---', 91)
((2, 2, 3, 3, 14, 4), '---', 52)
((2, 3, 3, 1, 14, 5), '---', 104)
((2, 1, 3, 3, 14, 5), '---', 104)
((2, 2, 2, 4, 14, 4), '---', 39)
((2, 1, 3, 4, 14, 4), '---', 78)
((2, 2, 2, 3, 14, 5), '---', 52)
((1, 2, 4, 3, 14, 4), '---', 78)
((2, 3, 1, 3, 14, 5), '---', 104)
((1, 3, 3, 3, 14, 4), '---', 52)
((1, 2, 3, 4, 14, 4), '---', 78)
((2, 1, 4, 3, 14, 4), '---', 78)
((2, 3, 1, 3, 13, 6), '---', 20)
((1, 2, 3, 4, 13, 5), '---', 24)
((2, 3, 3, 3, 13, 4), '---', 71)
((1, 2, 3, 5, 13, 4), '---', 12)
((2, 1, 4, 3, 13, 5), '---', 24)
((1, 3, 3, 4, 13, 4), '---', 12)
((2, 1, 5, 3, 13, 4), '---', 12)
((2, 3, 1, 4, 13, 5), '---', 24)
((

In [22]:
#check matches between r and s
v = {(2,3,3,12,4): 1}
s = sh(z4, v)
for ks in s:
    if ks in r:
        continue
    else:
        print ks

(2, 4, 3, 3, 12, 4)
(1, 4, 3, 4, 12, 4)
(2, 1, 4, 3, 12, 6)
(2, 2, 5, 3, 12, 4)
(3, 2, 4, 3, 12, 4)
(2, 2, 2, 3, 12, 7)
(2, 1, 4, 4, 12, 5)
(2, 1, 4, 5, 12, 4)
(3, 3, 3, 3, 12, 4)
(1, 2, 3, 4, 12, 6)
(2, 3, 3, 5, 8, 7)
(1, 2, 4, 4, 12, 5)
(1, 2, 5, 4, 12, 4)
(2, 2, 3, 5, 12, 4)
(2, 1, 3, 6, 12, 4)
(1, 2, 3, 6, 12, 4)
(1, 3, 3, 5, 12, 4)
(1, 3, 3, 4, 12, 5)
(2, 1, 5, 4, 12, 4)
(2, 3, 3, 6, 7, 7)
(1, 4, 4, 3, 12, 4)
(2, 3, 3, 8, 5, 7)
(2, 1, 3, 4, 12, 6)
(2, 3, 3, 3, 10, 7)
(2, 3, 3, 11, 2, 7)
(2, 3, 3, 7, 6, 7)
(2, 3, 3, 2, 11, 7)
(1, 2, 3, 5, 12, 5)
(2, 1, 3, 3, 12, 7)
(2, 1, 5, 3, 12, 5)
(2, 2, 2, 5, 12, 5)
(2, 2, 2, 4, 12, 6)
(1, 2, 5, 3, 12, 5)
(1, 5, 3, 3, 12, 4)
(2, 3, 3, 1, 12, 7)
(1, 2, 4, 5, 12, 4)
(2, 2, 4, 4, 12, 4)
(2, 3, 3, 9, 4, 7)
(2, 3, 1, 6, 12, 4)
(2, 2, 3, 4, 12, 5)
(1, 3, 3, 3, 12, 6)
(1, 2, 4, 3, 12, 6)
(2, 2, 2, 6, 12, 4)
(3, 2, 3, 3, 12, 5)
(2, 3, 1, 5, 12, 5)
(2, 3, 2, 5, 12, 4)
(4, 2, 3, 3, 12, 4)
(2, 2, 3, 3, 12, 6)
(2, 3, 3, 10, 3, 7)
(1, 3, 4, 3, 12, 5)
(2, 3

In [12]:
a = [0,1,3]; a[0:3]

[0, 1, 3]

In [13]:
u = {(2,3): 1/2}
v = {(2,): 1/3}
mono_st_new((3,1,7,9,6,3), (2,4,8,17))

{(2, 3, 1, 4, 7, 8, 9, 6, 3, 17): 1,
 (2, 3, 1, 4, 7, 8, 9, 6, 17, 3): 1,
 (2, 3, 1, 4, 7, 8, 9, 6, 20): 1,
 (2, 3, 1, 4, 7, 8, 9, 17, 6, 3): 1,
 (2, 3, 1, 4, 7, 8, 9, 23, 3): 1,
 (2, 3, 1, 4, 7, 8, 17, 9, 6, 3): 1,
 (2, 3, 1, 4, 7, 8, 26, 6, 3): 1,
 (2, 3, 1, 4, 7, 9, 6, 3, 8, 17): 1,
 (2, 3, 1, 4, 7, 9, 6, 8, 3, 17): 1,
 (2, 3, 1, 4, 7, 9, 6, 8, 17, 3): 1,
 (2, 3, 1, 4, 7, 9, 6, 8, 20): 1,
 (2, 3, 1, 4, 7, 9, 6, 11, 17): 1,
 (2, 3, 1, 4, 7, 9, 8, 6, 3, 17): 1,
 (2, 3, 1, 4, 7, 9, 8, 6, 17, 3): 1,
 (2, 3, 1, 4, 7, 9, 8, 6, 20): 1,
 (2, 3, 1, 4, 7, 9, 8, 17, 6, 3): 1,
 (2, 3, 1, 4, 7, 9, 8, 23, 3): 1,
 (2, 3, 1, 4, 7, 9, 14, 3, 17): 1,
 (2, 3, 1, 4, 7, 9, 14, 17, 3): 1,
 (2, 3, 1, 4, 7, 9, 14, 20): 1,
 (2, 3, 1, 4, 7, 17, 6, 3, 17): 1,
 (2, 3, 1, 4, 7, 17, 6, 17, 3): 1,
 (2, 3, 1, 4, 7, 17, 6, 20): 1,
 (2, 3, 1, 4, 7, 17, 17, 6, 3): 1,
 (2, 3, 1, 4, 7, 17, 23, 3): 1,
 (2, 3, 1, 4, 8, 7, 9, 6, 3, 17): 1,
 (2, 3, 1, 4, 8, 7, 9, 6, 17, 3): 1,
 (2, 3, 1, 4, 8, 7, 9, 6, 20): 1,
 (2, 3, 1, 4

In [1]:
import data

In [2]:
data.extend_del_x(7)

{'yxxxxxx': 1, 'yxxxxyx': 1, 'yxxxyxx': 1, 'yxxxyyx': 1, 'yxxyxxx': 1, 'yxxyxyx': 1, 'yxxyyxx': 1, 'yxxyyyx': 1, 'yxyxxxx': 1, 'yxyxxyx': 1, 'yxyxyxx': 1, 'yxyxyyx': 1, 'yxyyxxx': 1, 'yxyyxyx': 1, 'yxyyyxx': 1, 'yxyyyyx': 1, 'yyxxxxx': 1, 'yyxxxyx': 1, 'yyxxyxx': 1, 'yyxxyyx': 1, 'yyxyxxx': 1, 'yyxyxyx': 1, 'yyxyyxx': 1, 'yyxyyyx': 1, 'yyyxxxx': 1, 'yyyxxyx': 1, 'yyyxyxx': 1, 'yyyxyyx': 1, 'yyyyxxx': 1, 'yyyyxyx': 1, 'yyyyyxx': 1, 'yyyyyyx': 1},
{'yxxxxxxx': 1, 'yxxxxxyx': 1, 'yxxxxyxx': 1, 'yxxxxyyx': 1, 'yxxxyxxx': 1, 'yxxxyxyx': 1, 'yxxxyyxx': 1, 'yxxxyyyx': 1, 'yxxyxxxx': 1, 'yxxyxxyx': 1, 'yxxyxyxx': 1, 'yxxyxyyx': 1, 'yxxyyxxx': 1, 'yxxyyxyx': 1, 'yxxyyyxx': 1, 'yxxyyyyx': 1, 'yxyxxxxx': 1, 'yxyxxxyx': 1, 'yxyxxyxx': 1, 'yxyxxyyx': 1, 'yxyxyxxx': 1, 'yxyxyxyx': 1, 'yxyxyyxx': 1, 'yxyxyyyx': 1, 'yxyyxxxx': 1, 'yxyyxxyx': 1, 'yxyyxyxx': 1, 'yxyyxyyx': 1, 'yxyyyxxx': 1, 'yxyyyxyx': 1, 'yxyyyyxx': 1, 'yxyyyyyx': 1, 'yyxxxxxx': 1, 'yyxxxxyx': 1, 'yyxxxyxx': 1, 'yyxxxyyx': 1, 'yyxxyxxx

In [3]:
data.del_x

[{},
 {'yx': 1},
 {'yxx': 1, 'yyx': 1},
 {'yxxx': 1, 'yxyx': 1, 'yyxx': 1, 'yyyx': 1},
 {'yxxxx': 1,
  'yxxyx': 1,
  'yxyxx': 1,
  'yxyyx': 1,
  'yyxxx': 1,
  'yyxyx': 1,
  'yyyxx': 1,
  'yyyyx': 1},
 {'yxxxxx': 1,
  'yxxxyx': 1,
  'yxxyxx': 1,
  'yxxyyx': 1,
  'yxyxxx': 1,
  'yxyxyx': 1,
  'yxyyxx': 1,
  'yxyyyx': 1,
  'yyxxxx': 1,
  'yyxxyx': 1,
  'yyxyxx': 1,
  'yyxyyx': 1,
  'yyyxxx': 1,
  'yyyxyx': 1,
  'yyyyxx': 1,
  'yyyyyx': 1}]

In [8]:
d = mono_sh_new('yxx','yx'); d

{'yxxyx': 1, 'yxyxx': 3, 'yyxxx': 6}

In [17]:
d = mono_diff_del(2, 'yxx'); d

{'yxxxx': -1, 'yxyxx': 1, 'yxyyx': 1, 'yyxxx': 0, 'yyyxx': 1}

In [18]:
d = mono_diff_del(2, 'yyx'); d

{'yxxyx': -1, 'yyxxx': -1, 'yyxyx': -1, 'yyyxx': 0, 'yyyyx': 1}

In [19]:
mono_diff_del(3, 'yx')

{'yxxxx': -1, 'yxyxx': -1, 'yyxxx': 0, 'yyxyx': 1, 'yyyxx': 0, 'yyyyx': 1}

In [32]:
xyDict_to_list(5, mono_diff_del(3, 'yx'))

[-1, 0, -1, 0, 0, 1, 0, 1]

In [4]:
A = []
A.append(xyDict_to_list(5, mono_diff_del(1, 'yxxx'))) #dual pair 1
A.append(xyDict_to_list(5, mono_diff_del(1, 'yxyx'))) 
A.append(xyDict_to_list(5, mono_diff_del(1, 'yyxx')))
A.append(xyDict_to_list(5, mono_diff_del(1, 'yyyx'))) #dual pair 1
A.append(xyDict_to_list(5, mono_diff_del(2, 'yxx'))) #dual pair 2
A.append(xyDict_to_list(5, mono_diff_del(2, 'yyx'))) #dual pair 2
A.append(xyDict_to_list(5, mono_diff_del(3, 'yx')))
MA = Matrix(QQ, A)
MA

[-1  1  1  0  1  0  0  0]
[ 0 -1 -1  1  0  1  0  0]
[ 0  0 -1  0 -1  1  1  0]
[ 0  0  0 -1  0 -1 -1  1]
[-1  0  1  1  0  0  1  0]
[ 0 -1  0  0 -1 -1  0  1]
[-1  0 -1  0  0  1  0  1]

In [5]:
kernel(MA)

Vector space of degree 7 and dimension 2 over Rational Field
Basis matrix:
[ 1  0  0 -1 -1  1  0]
[ 0  1  1  2  1 -1 -1]

In [6]:
Y = vector([1,0,0,0,0,0,0,-1]) #to derive dual between yxxxx & yyyyx
X = MA.solve_left(Y)
X

(-1/2, -1/2, -1/2, -1, -1/2, 0, 0)

In [7]:
Y = vector([0,1,1,1,1,1,1,-2]) #mzv(1,2)的外帶定理
X = MA.solve_left(Y)
X

(1/2, -1/2, -1/2, -2, -1/2, 0, 0)

In [62]:
X + 1/2*vector([0,1,1,2,1,-1,-1]) 

(-1/2, 0, 0, 0, 0, -1/2, -1/2)

In [65]:
Y = vector([0,1,0,-1,0,0,0,0]) #to derive dual between yxxyx (col 1) & yxyyx (col 3)
X1 = MA.solve_left(Y)
X1

(1/2, -1/2, 1/2, 0, -1/2, 0, 0)

In [66]:
Y = vector([0,0,1,0,0,-1,0,0]) #to derive dual between yxyxx (col 2) & yyxyx (col 5)
X2 = MA.solve_left(Y)
X2 

(-1/2, -1/2, -1/2, 0, 1/2, 0, 0)

In [67]:
X1 + X2 #comes from the second row (row 1) of MA

(0, -1, 0, 0, 0, 0, 0)

In [44]:
import time
u = {(1,2,3,4,5,6,7,8,9): 1}
v = {(9,8,7,6,5,4,3): 1}
t0 = time.time()
w = st_new(u,v)
t1 = time.time()
print(t1-t0)
#st: 3.3
#st_new: 13.3
#st_new 反而比較慢

13.5475680828


{(1, 2): 1}